# Genetic Algorithms Lecture 5, Now Also In English!

Last lecture mentioned **misleading problems** (*zwodnicze*)

We have $f_{11} > f_{00}, f_{01}, f_{10}$ - the 11 version is more optimal than all others.

We also have $f_{x1} < f_{x0}$ - on average, those with 0 in the second place are better than those with 1 in the second place, **even though** $11$ is the optimal candidate.

### Creation and destruction of chromosomes
Consider the crossing over of these, swapping their first digits:

| #  | 00  | 01  | 10  | 11  |
|----| --- | --- | --- | --- |
| 00 |same|same, 00 01 |same |01 10|
| 01 |same|same |11 00 |same |
| 10 |same|00 11 |same |same |
| 11 |01 10 |same |same |same |

Probability of seeing 00 in the next iteration, neglecting mutation:
\begin{equation}
p_{00}^{t+1} = p_{00}^t \frac{f_{00}}{\bar{f}} (1-p_c' \frac{f_{11}}{\bar{f}} p_{11}^t) + p_c' \frac{f_{01} f_{10}}{\bar{f}^2} p_{01}^t p_{10}^t
\end{equation}

The crossover probability is
\begin{equation}
p_c' = \frac{p_c S(H)}{l-1}
\end{equation}
where $S(H)$ is the chromosome span or spread (*rozpietosc*) and $l$ is the length of the chromosome

The $\frac{f_{11}}{\bar{f}} p_{11}^t$ part is the probability of 11 propagating over from the previous iteration. The whole thing comes from the fact that crossing over destroys 00 if it interacts with 11 (see table).

The final part $p_c' \frac{f_{01} f_{10}}{\bar{f}^2}$ results from the similar fact: 01 meeting 10 would also produce 00! We might need a factor of 2 there due to distinguishing between (01, 10) and (10, 01), but Goldberg didn't have that.

The others would look like:
\begin{equation}
p_{01}^{t+1} = p_{01}^t \frac{f_{01}}{\bar{f}} (1-p_c' \frac{f_{10}}{\bar{f}} p_{10}^t) + p_c' \frac{f_{00} f_{11}}{\bar{f}^2} p_{00}^t p_{11}^t
\end{equation}

\begin{equation}
p_{10}^{t+1} = p_{10}^t \frac{f_{10}}{\bar{f}} (1-p_c' \frac{f_{01}}{\bar{f}} p_{01}^t) + p_c' \frac{f_{00} f_{11}}{\bar{f}^2} p_{00}^t p_{11}^t
\end{equation}

\begin{equation}
p_{11}^{t+1} = p_{11}^t \frac{f_{11}}{\bar{f}} (1-p_c' \frac{f_{00}}{\bar{f}} p_{00}^t) + p_c' \frac{f_{01} f_{10}}{\bar{f}^2} p_{01}^t p_{10}^t
\end{equation}

**Complementary schemes** destroy each other and produce new schemes on crossing over. 00 and 11 are a pair, 01 and 10 is another.

### Numerical simulations of misleading scenarios

For a misleading problem of type 1 (**or 2**), if all probabilities start out distributed initially, for a plot of $p(t)$ we'd get:
* $p_{01}$ shifted Maxwellian (starting at $0.25$ due to uniform initial distribution
* $p_{00}, p_{10}$ dying out quickly 
* $p_{11}$ falling a bit initially, then asymptotically going to 1.

Note that there's a phase where the best scheme 11 is losing to 01! The second best is produced by the interaction of 00 and 11. If 00 still exists, 01 will be produced. However, 00 is the worst and is going to be dying out quickly, so the 'reaction substrate' will be depleting quickly.

\begin{equation}
p_{00}^{t=0} > 0 \implies p_{11}^{t=\inf} = 1
\end{equation}

For a misleading problem of type 2, 00 was the second best, not the worst. If we have an initial distribution like
* 0.5 00
* 0.166 10, 11, 10

The second best, $p_{00} \to 1$. 

For misleading problems of type 2, the complementary (to 11) schema 00 can dominate the population if there's enough of it in the initial population. How much exactly? That depends on the values of $f_{11}, f_{00}, f_{01}, f_{10}$.

We say that a genetic algorithm has been misled if the optimal schema is not the most represented in the final population.

### A 3D case
For a 3D case (where our sequences are binary 3-tuples) we have a cube instead of a square.

(0,x,0) would be a schema of order 2 and represented by 2 points, therefore a line going through (0,1,0), (0,0,0).

Each schema of order
* 3 is a point.
* 2 is a line.
* 1 is a plane, perpendicular to the axes. You can't have a more complex surface (say, at an angle) with a a binary (0 or 1) genome!
* 0 (xxx) is the entire space.

In general, hypersurfaces in various dimensionalities can be used to visualize and describe genes. Thus you can use linear algebra and the like in genetic algorithms.

## Transforming the goal function
(by the goal function I mean $f_{00}$ and the like).

It often happens that our goal function has negative values. How do we check that? Use a vector of N random values $x_i^{t=0}$ or $\vec{x}$ and take $ c = \min \vec{x}$. Our new cost function $f'(x) = f(x) + |c| * 1.2$ (we use 1.2 or another numerical factor because we hit exactly 0 via sampling the function f, we want to have some leeway).

In practice you do this at every iteration!

Note that we want **linear scaling**, taking, say, $f(x)^2$ would distort our population selection (which "by default" is linear), though it solves the problem of negative values, it messes up in other places.

### Tournament selection
Suppose however that it's difficult to "renormalize" the function like this. There's an alternative approach called *tournament selection* (*selekcja turniejowa*).

We have a preset population, we pick, say, 4 members of that set. We apply the selection operator to the 4 Chosen Ones and pick the 1 or 2 best ones.

We do this again for another random 4 members of the set (the randomness is in picking the elements). Note that we can still pick the Best Chosen One from the previous trial (or trials).



If we want to minimize instead of maximize a function, take $f'(x) = -f(x)$.

### Time scaling of the goal function
(skalowanie czasowe)

In the beginning use a loose rather than a strict selection (say, add a positive constant to the goal function) - this prevents premature settling down in a local (rather than global) maximum.

In the end, selection should be stricter - if our entire population is near a global maximum, selection will be slow (as it's based on the differences between function values evaluated at members of the population). In practice you SUBTRACT a constant $f'(x) = f(x) - d$. The constant d should be slightly below all the representatives.

* There is a beautiful algorithm called simulated annealing that uses this principle, for which these notes are too small to contain. Wikipedia has more on that. *

In general $f'(x) = af(x) + b$. $a$ only matters if $b \neq 0$.